<a href="https://colab.research.google.com/github/aleksl17/IKT440ptmc/blob/colab/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pycuda
!pip3 install pyTsetlinMachineCUDA
!pip3 install opencv-python

from PyTsetlinMachineCUDA.tm import MultiClassConvolutionalTsetlinMachine2D
#import sys
#sys.path.append('../pyTsetlinMachine/')
#from tm import MultiClassConvolutionalTsetlinMachine2D
import numpy as np
from time import time
import copy
import matplotlib.pyplot as plt
import math
import imgaug


from keras.datasets import cifar10

import cv2

factor = 1
clauses = 2000*factor
T = 75*100*factor
s = 10.0
patch = 8
img_size = 32

epochs = 20

labels = [b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']

tm = MultiClassConvolutionalTsetlinMachine2D(clauses, T, s, (patch, patch), number_of_state_bits=8, max_weight=255)
#tm = MultiClassConvolutionalTsetlinMachine2D(clauses, T, s, (patch, patch), weighted_clauses=True)

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

#do augmentation here
# Global Variables
augmentation_percentage = 30

# Import and manipulate dataset
a_p = math.floor((augmentation_percentage / 100) * len(X_train))
x_train_aug_before = copy.deepcopy(X_train[0:a_p])
x_train_aug_before = x_train_aug_before.astype("float32")

# Augmentation variables
seq = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(rotate=(-25, 25)),
    imgaug.augmenters.SaltAndPepper(p=(0, 0.1)),
    imgaug.augmenters.Crop(percent=(0, 0.25))
])

# Image augmentation
x_aug = seq.augment_images(x_train_aug_before)
y_aug = Y_train[0:a_p]

#do adaptive gaussian filtering here

def gaussian_kernel(size, sigma=1):
  size = int(size) // 2
  x, y = np.mgrid[-size:size+1, -size:size+1]
  normal = 1 / (2.0 * np.pi * sigma**2)
  g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
  return g

gaussian_images = []

for img in x_aug:
  new_img = cv2.filter2D(img, -1, gaussian_kernel(2,sigma=1))
  gaussian_images.append(new_img)

gaussian_images = np.array(gaussian_images)
gaussian_images = gaussian_images.astype(int)


#append augmented images to training data
X_train = np.concatenate((X_train, gaussian_images), axis=0)
Y_train = np.concatenate((Y_train, y_aug),axis=0)

#reshape dataset
Y_train=Y_train.reshape(Y_train.shape[0])
Y_test=Y_test.reshape(Y_test.shape[0])

accuracy_history = []

print("\nAccuracy over %d epochs:\n" % epochs)
for i in range(epochs):
        start_training = time()
        tm.fit(X_train, Y_train, epochs=1, incremental=True)
        stop_training = time()

        start_testing = time()
        result_test = 100*(tm.predict(X_test) == Y_test).mean()
        accuracy_history.append(result_test)
        result_train= 100*(tm.predict(X_train) == Y_train).mean()
        stop_testing = time()

        print("#%d Accuracy: %.2f%%/%.2f%% Training: %.2fs Testing: %.2fs" % (i+1, result_train, result_test, stop_training-start_training, stop_testing-start_testing))


plt.plot(accuracy_history)
plt.show()

#testing

pred = tm.predict(X_test)

correct = 0
fail = 0
for i in range(len(X_test)):
  if pred[i] == Y_test[i]:
    correct += 1
  else:
    fail += 1

print("Testing Accuracy after "+ str(epochs) +" epochs:")
print(str(correct/(correct+fail)))




Accuracy over 20 epochs:

#1 Accuracy: 12.10%/12.61% Training: 212.92s Testing: 183.85s
#2 Accuracy: 11.83%/11.98% Training: 205.93s Testing: 184.70s
#3 Accuracy: 12.16%/12.59% Training: 206.93s Testing: 186.70s
#4 Accuracy: 11.98%/12.21% Training: 207.20s Testing: 187.78s
#5 Accuracy: 12.26%/12.54% Training: 206.30s Testing: 186.23s
#6 Accuracy: 12.49%/12.68% Training: 205.93s Testing: 184.76s
#7 Accuracy: 12.06%/12.36% Training: 206.29s Testing: 188.48s
